<a href="https://colab.research.google.com/github/anshuraj460016/AI-Lab/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
list = [10, 20, 4, 45, 99]
list.sort()
print("Sorted list:", list)

Sorted list: [4, 10, 20, 45, 99]


In [19]:
list = [10, 20, 30, 40, 50]
sum = 0
for num in list1:
  sum += num
print("Sum of all elements in the list:", sum)

Sum of all elements in the list: 150


In [22]:
string = input("Enter a string: ")
if string == string[::-1]:
  print("The string is a palindrome")
else:
  print("The string is not a palindrome")

Enter a string: 6
The string is a palindrome


In [23]:
num1 = int(input("Enter the first number: "))
num2 = int(input("Enter the second number: "))
sum = num1 + num2
print("The sum of the numbers is:", sum)

Enter the first number: 4
Enter the second number: 6
The sum of the numbers is: 10


In [24]:
print("Hello, World!")

Hello, World!


In [25]:
num = int(input("Enter a non-negative integer: "))
factorial = 1
if num < 0:
  print("Factorial does not exist for negative numbers")
elif num == 0:
  print("The factorial of 0 is 1")
else:
  for i in range(1,num + 1):
    factorial = factorial*i
  print("The factorial of",num,"is",factorial)

Enter a non-negative integer: 6
The factorial of 6 is 720


In [26]:


def is_prime(n):
  """Checks if a number is prime."""
  if n <= 1:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      return False
  return True

def check_number(num):
  """Checks if a number is odd, even, or prime and prints the result."""

  if num % 2 == 0:
    print(f"{num} is even.")
  else:
    print(f"{num} is odd.")

  if is_prime(num):
    print(f"{num} is prime.")

user_input = int(input("Enter an integer: "))
check_number(user_input)

Enter an integer: 6
6 is even.


In [27]:
rows = 5
for i in range(1, rows+1):
    for j in range(1, i+1):
        print("*", end="")
    print()

*
**
***
****
*****


In [28]:
sum = 0
for num in range(2, 101, 2):
    sum += num
print("Sum of even numbers:", sum)

Sum of even numbers: 2550


In [29]:
def sum_of_digits(number):

  sum = 0
  while number > 0:
    digit = number % 10
    sum += digit
    number //= 10
  return sum

num = int(input("Enter a number: "))
result = sum_of_digits(num)
print("The sum of digits of", num, "is", result)

Enter a number: 678
The sum of digits of 678 is 21
